In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CHANNELS=3

train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain Tumar Data set/Dataset/New folder (2)/Train',
    batch_size=32,
    class_mode='categorical',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
   '/content/drive/MyDrive/Brain Tumar Data set/Dataset/New folder (2)/Validation',
    shuffle=False,
    class_mode='categorical',
    target_size=(224,224))
test_generator = validation_datagen.flow_from_directory(
   '/content/drive/MyDrive/Brain Tumar Data set/Dataset/New folder (2)/Test',
    shuffle=False,
    class_mode='categorical',
    target_size=(224,224))

Found 2180 images belonging to 3 classes.
Found 306 images belonging to 3 classes.
Found 629 images belonging to 3 classes.


In [ ]:
x,y=test_generator.next()
x.shape

(32, 224, 224, 3)

In [ ]:
VGG19 = tf.keras.applications.VGG19(
    include_top=False, 
    weights = 'imagenet', 
    input_shape=(224,224,3),
    )

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
VGG19.trainable = False

In [ ]:
model = tf.keras.models.Sequential([
  VGG19,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.8),
  tf.keras.layers.Dense(units=1024, activation='relu'),
  tf.keras.layers.Dense(units=512, activation='relu'),
  tf.keras.layers.Dense(units=3, activation='softmax'), 
])

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    batch_size=32,
    validation_data=test_generator,
    epochs=10
)

Epoch 1/10
69/69 [==============================] - 1753s 25s/step - loss: 5.4800 - accuracy: 0.6670 - val_loss: 1.2171 - val_accuracy: 0.7615
Epoch 2/10
69/69 [==============================] - 1703s 25s/step - loss: 0.7559 - accuracy: 0.7950 - val_loss: 0.5973 - val_accuracy: 0.7949
Epoch 3/10
69/69 [==============================] - 1711s 25s/step - loss: 0.5045 - accuracy: 0.8335 - val_loss: 1.1601 - val_accuracy: 0.7250
Epoch 4/10
69/69 [==============================] - 1709s 25s/step - loss: 0.4260 - accuracy: 0.8573 - val_loss: 0.7309 - val_accuracy: 0.7409
Epoch 5/10
69/69 [==============================] - 1705s 25s/step - loss: 0.3648 - accuracy: 0.8683 - val_loss: 0.6070 - val_accuracy: 0.7790
Epoch 6/10
69/69 [==============================] - 1717s 25s/step - loss: 0.3311 - accuracy: 0.8794 - val_loss: 0.7328 - val_accuracy: 0.7424
Epoch 7/10
69/69 [==============================] - 1730s 25s/step - loss: 0.3077 - accuracy: 0.8894 - val_loss: 0.5593 - val_accuracy: 0.7917

In [ ]:
loss, accuracy = model.evaluate(test_generator )

# Print the accuracy
print("Accuracy: " + str(accuracy))
# Print the loss
print("Loss: " + str(loss))

20/20 [==============================] - 383s 19s/step - loss: 0.7143 - accuracy: 0.7806
Accuracy: 0.7806041240692139
Loss: 0.7142829298973083


In [ ]:
import numpy as np
from sklearn import metrics

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis = 1)
true_classes = test_generator.classes
class_labels = ['glioma', 'meningioma', 'pituitary_tumor']
report = metrics.classification_report(true_classes, predicted_classes, target_names = class_labels)
print(report)

20/20 [==============================] - 381s 19s/step
                 precision    recall  f1-score   support

         glioma       0.86      0.82      0.84       291
     meningioma       0.64      0.56      0.60       152
pituitary_tumor       0.77      0.90      0.83       186

       accuracy                           0.78       629
      macro avg       0.76      0.76      0.75       629
   weighted avg       0.78      0.78      0.78       629



In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm = metrics.confusion_matrix(true_classes, predicted_classes)
df_cm = pd.DataFrame(cm, index = [i for i in ['glioma', 'meningioma', 'pituitary_tumor']],
                  columns = [i for i in ['glioma', 'meningioma', 'pituitary_tumor']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, fmt='d')

NameError: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model accuacy")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy', 'validation accuracy'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'], loc='upper right')
plt.show()